In [1]:
import cephes4py
from scipy.special import bdtr as scipy_btdr, smirnov as scipy_smirnov, fdtr as scipy_fdtr, ellipe as scipy_ellipe, jv as scipy_jv

Let's time the SciPy Binomial Distribution function:

In [2]:
%timeit scipy_btdr(4, 6, 0.3)


1.72 µs ± 101 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


Okay, running the SciPy version takes 1.72 microseconds. That's practically nothing, surely we can't beat that?

Hold my beer.

In [3]:
%timeit cephes4py.bdtr(4, 6, 0.3)


303 ns ± 17.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


Running the cephes4py version takes 303 nanoseconds, or 0.303 microseconds. An 82% decrease in execution time!

Let's do some more tests:

In [4]:
%timeit scipy_fdtr(1, 2, 1)

2.27 µs ± 384 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [5]:
%timeit cephes4py.fdtr(1, 2, 1)

337 ns ± 16.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [6]:
a = 3.5
b = 2.1
e_sq = 1.0 - b**2/a**2
%timeit scipy_ellipe(e_sq)

763 ns ± 102 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [7]:
%timeit cephes4py.ellpe(e_sq)

233 ns ± 18 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [8]:
v = 1
z = 1.0
%timeit scipy_jv(v, z)

1.71 µs ± 130 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [9]:
%timeit cephes4py.jv(v, z)

327 ns ± 53.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [10]:
n = 5
p = 0.5
%timeit scipy_smirnov(n, p)

3.8 µs ± 191 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [11]:
%timeit cephes4py.smirnov(n, p)

381 ns ± 21.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


Moreover, you cannot use SciPy's special functions in Numba-jitted functions in nopython mode, which might introduce a bottleneck to your code. cephes4py functions, on the other hand, can. Look:

In [12]:
import numba as nb

@nb.njit
def test_nb_scipy():
    n = 5
    p = 0.5
    return scipy_smirnov(n, p)

test_nb_scipy()

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'scipy_smirnov':[0m [1m[1mCannot determine Numba type of <class 'numpy.ufunc'>[0m
[1m
File "../../tmp/ipykernel_4867/159472629.py", line 7:[0m
[1m<source missing, REPL/exec in use?>[0m
[0m

In [13]:
@nb.njit
def test_nb_cephes4py():
    n = 5
    p = 0.5
    return cephes4py.smirnov(n, p)

test_nb_cephes4py()

0.05600000000000001